In [1]:
import pandas as pd
import numpy as np
trainingSetDF = pd.read_excel("DrugInformation_ImmuneModPaper.xlsx")
###Drop duplicates (certain drugs listed multiple times with slightly different smiles)
trainingSetDF = trainingSetDF.drop_duplicates(subset = "CHEMBL_ID")
##Remove compounds without ChEMBL IDs
trainingSetDF = trainingSetDF[~pd.isna(trainingSetDF.CHEMBL_ID)]
drugBankDF = pd.read_csv("structure links_1.csv")
##Remove compounds without ChEMBL IDs
drugBankDF = drugBankDF[~pd.isna(drugBankDF.CHEMBL_ID)]
How many drugs overlap between the training set and drugbank?
overlap = len(set(trainingSetDF.CHEMBL_ID.unique()).intersection(set(drugBankDF.CHEMBL_ID.unique())))
print(overlap)
##Merge
combinedDF = pd.merge(trainingSetDF, drugBankDF, on= "CHEMBL_ID", how = "inner")
assert combinedDF.shape[0] == overlap

Object `drugbank` not found.
747


In [3]:
combinedDF.head()

,SMILES_x,CHEMSPIDER_ID,CHEMBL_ID,CHEMBL_KNOWN_DRUG,CHEMBL_NAME,CHEMBL_ALTERNATE_NAME,CLINICALTRIALS_DB,CONCENTRATION_uM,DrugBank ID,Name,...,SMILES_y,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,HET ID,ChemSpider ID,BindingDB ID
0,BrC1=CN([C@@H]2O[C@H](CO)[C@@H](O)C2)C(=O)NC1=O,5813.0,CHEMBL222280,No,BROXURIDINE,SID50105447|SID17389223|SID90341500|SID5646351...,1,10,DB12028,Broxuridine,...,[H][C@]1(O)C[C@@]([H])(O[C@]1([H])CO)N1C=C(Br)...,C9H11BrN2O5,NaN,NaN,6035.0,347828343.0,472552.0,U33,5813.0,50207303.0
1,Brc1c2nccnc2ccc1NC=1NCCN=1,2341.0,CHEMBL844,Yes,BRIMONIDINE,Alphagan|Brimonidine|UK-14304|Alphagan-P|SID10...,256,10,DB00484,Brimonidine,...,BrC1=C(NC2=NCCN2)C=CC2=NC=CN=C12,C11H10BrN5,C07886,D07540,2435.0,46507305.0,3175.0,NaN,2341.0,34572.0
2,Brc1cc(cc(Br)c1O)C(=O)c1c2c(oc1CC)cccc2,2243.0,CHEMBL388590,No,BENZBROMARONE,SID11110651|SID26664708|SID24424558|SID5010399...,2,10,DB12319,Benzbromarone,...,CCC1=C(C(=O)C2=CC(Br)=C(O)C(Br)=C2)C2=CC=CC=C2O1,C17H12Br2O3,NaN,D01056,2333.0,347828582.0,3023.0,NaN,2243.0,50158460.0
3,Brc1ccc(cc1)[C@H](CCN(C)C)c1ncccc1,16068.0,CHEMBL1201287,Yes,DEXBROMPHENIRAMINE,SID90341837|Dexbrompheniramine|SID29216402|,0,10,DB00405,Dexbrompheniramine,...,CN(C)CC[C@@H](C1=CC=C(Br)C=C1)C1=CC=CC=N1,C16H19BrN2,NaN,NaN,16960.0,46505186.0,59269.0,NaN,16068.0,NaN
4,Brc1ccccc1C[N+](CC)(C)C,2337.0,CHEMBL1199080,Yes,BRETYLIUM,SID50113155|Bretylium|SID50113154|,0,17,DB01158,Bretylium,...,CC[N+](C)(C)CC1=CC=CC=C1Br,C11H17BrN,C06855,D00645,2431.0,46505320.0,3172.0,NaN,2337.0,NaN


In [4]:
from rdkit import Chem
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D = True)

In [5]:
mols = [Chem.MolFromSmiles(smi) for smi in combinedDF['SMILES_x']]

In [7]:
calDF = calc.pandas(mols)

 28%|██▊       | 207/747 [00:30<02:20,  3.84it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 30%|██▉       | 224/747 [00:33<02:48,  3.10it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 41%|████▏     | 310/747 [00:47<01:47,  4.06it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 747/747 [01:58<00:00,  6.29it/s]


In [8]:
calDF.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,13.003721,11.805899,0,0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505,26,88.0,104.0,7.027778,3.777778
1,13.423098,11.209659,0,3,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515,24,90.0,106.0,4.305556,3.750000
2,17.125041,14.795147,0,0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988,38,118.0,143.0,8.000000,4.861111
3,14.389425,12.216867,0,1,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718,25,92.0,103.0,6.305556,4.305556
4,9.579175,9.100648,0,1,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248,16,62.0,68.0,5.784722,2.944444


In [42]:
calDF.info()

<class 'mordred._base.pandas_module.MordredDataFrame'>
RangeIndex: 747 entries, 0 to 746
Columns: 1613 entries, ABC to mZagreb2
dtypes: float64(1611), object(2)
memory usage: 9.2+ MB


In [43]:
info_return = calDF.info()
count_return = calDF.count()
info_return is None

<class 'mordred._base.pandas_module.MordredDataFrame'>
RangeIndex: 747 entries, 0 to 746
Columns: 1613 entries, ABC to mZagreb2
dtypes: float64(1611), object(2)
memory usage: 9.2+ MB


True

In [44]:
type(count_return)

pandas.core.series.Series

In [45]:
calDF_missing = calDF.isna()

In [46]:
calDF_missing.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [47]:
calDF_num_missing = calDF_missing.sum()

In [48]:
calDF_num_missing

ABC           0
ABCGG         0
nAcid       595
nBase       433
SpAbs_A       0
           ... 
WPol          1
Zagreb1       0
Zagreb2       0
mZagreb1      0
mZagreb2      0
Length: 1613, dtype: int64

In [40]:
calDF = calDF.replace({'0':np.nan, 0:np.nan})

In [49]:
len(calDF)

747

In [72]:
calDF.columns[calDF.isnull().mean() < .05]

Index(['ABC', 'ABCGG', 'SpAbs_A', 'SpMax_A', 'SpDiam_A', 'SpAD_A', 'SpMAD_A',
       'LogEE_A', 'VE1_A', 'VE2_A',
       ...
       'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2', 'Count'],
      dtype='object', length=922)

In [75]:
calDF_missing = calDF[calDF.columns[calDF.isnull().mean() < .05]]

In [76]:
calDF_missing

,ABC,ABCGG,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Count
0,13.003721,11.805899,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,3.772584,0.221917,...,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778,1144
1,13.423098,11.209659,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,3.719596,0.218800,...,63.007560,291.011957,10.778221,515.0,24.0,90.0,106.0,4.305556,3.750000,1144
2,17.125041,14.795147,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,4.079534,0.185433,...,70.803080,421.915318,12.409274,988.0,38.0,118.0,143.0,8.000000,4.861111,1154
3,14.389425,12.216867,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,3.842414,0.202232,...,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556,1121
4,9.579175,9.100648,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,3.345743,0.257365,...,43.893749,242.053888,8.068463,248.0,16.0,62.0,68.0,5.784722,2.944444,1096
5,23.598392,18.449570,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,5.024165,0.167472,...,79.445461,449.118544,8.636895,2633.0,49.0,160.0,190.0,9.722222,6.527778,1176
6,30.079292,22.385369,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,4.737849,0.118446,...,76.538683,536.438202,5.587898,7984.0,57.0,194.0,216.0,16.013889,8.888889,1074
7,26.472550,20.542158,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,4.438453,0.130543,...,70.685296,476.223056,7.107807,3458.0,60.0,180.0,215.0,10.902778,7.486111,1158
8,21.867940,17.016372,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,4.298230,0.148215,...,63.939923,405.185942,7.108525,2488.0,43.0,142.0,163.0,8.527778,6.777778,1122
9,3.932653,4.244375,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,2.309401,0.384900,...,30.698690,127.943185,15.992898,29.0,4.0,22.0,21.0,4.222222,1.444444,741


1124.0307898259705